# Bitcoin Options Analysis - Concise Version

**Streamlined put-call parity regression analysis** with YAML configuration.

## Key Features
- **Configuration-Driven**: All parameters from `config/base_offset_config.yaml`
- **Rate Extraction**: USD rate (r) and BTC rate (q) from options pricing  
- **Constrained Optimization**: No-arbitrage forward pricing
- **Exponential Smoothing**: Rate differential smoothing
- **Interactive Visualization**: Multi-panel time series plots

*All cells optimized to ≤40 lines for improved readability* ✅

## 1. Setup & Configuration 🔧

In [ ]:
# Essential Imports & Setup 📦
import polars as pl, numpy as np, plotly.graph_objects as go, os, sys, warnings
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from IPython.display import display, HTML
import plotly.offline as offline, plotly.io as pio

# Project setup 📁
current_dir = os.getcwd()
project_root = os.path.dirname(current_dir) if current_dir.endswith('notebooks') else current_dir
if project_root not in sys.path: sys.path.append(project_root)

# Import project modules 🔗
from utils.market_data.orderbook_deribit_md_manager import OrderbookDeribitMDManager
from utils.market_data.deribit_md_manager import DeribitMDManager
from utils.base_offset_fitter.weight_least_square_regressor import WLSRegressor
from utils.base_offset_fitter.nonlinear_minimization import NonlinearMinimization
from utils.base_offset_fitter.fitter_result_manager import FitterResultManager
from utils.base_offset_fitter.maths import convert_rate_into_parameter
from utils.reporting.html_table_generator import generate_price_comparison_table, calculate_tightening_stats, print_tightening_effectiveness
from utils.reporting.plotly_manager import PlotlyManager

# Configure libraries ⚙️
pl.Config.set_tbl_rows(10)
pio.renderers.default = "notebook"
warnings.filterwarnings('ignore')
offline.init_notebook_mode()

print("✅ All libraries imported successfully!")

In [ ]:
# Load Configuration 📋
from config.config_loader import load_config

config = load_config(os.path.join(project_root, 'config', 'base_offset_config.yaml'))

# Extract key variables 🔑
date_str = config.date_str
use_orderbook_data = config.use_orderbook_data
conflation_every = config.conflation_every
conflation_period = config.conflation_period
use_constrained_optimization = config.use_constrained_optimization
time_interval_seconds = config.time_interval_seconds
old_weight = config.old_weight
rate_constraints = config.get_rate_constraints()

print(f"✅ Config loaded | 📅 Date: {date_str} | 📊 Source: {'OrderBook' if use_orderbook_data else 'BBO'}")
print(f"🔧 Conflation: {conflation_every}/{conflation_period} | ⚙️ Constrained: {use_constrained_optimization}")
print(f"📊 Constraints: r∈[{rate_constraints['r_min']:.1%},{rate_constraints['r_max']:.1%}], q∈[{rate_constraints['q_min']:.1%},{rate_constraints['q_max']:.1%}]")

In [ ]:
# Data Loading & Pipeline Setup 🚀
data_file = os.path.join(project_root, config.get_data_file_path())
print(f"📂 Loading: {data_file}")

df_raw = pl.scan_csv(data_file)
if use_orderbook_data: df_raw = df_raw.with_columns(pl.col('index_price').cast(pl.Float64))

# Initialize managers 🏗️
if use_orderbook_data:
    symbol_manager = OrderbookDeribitMDManager(df_raw, date_str, config)
    print("🔧 Using OrderBook manager")
else:
    symbol_manager = DeribitMDManager(df_raw, date_str)
    print("🔧 Using BBO manager")

# Initialize fitters 🧮
wls_regressor = WLSRegressor(symbol_manager, config)
nonlinear_minimizer = NonlinearMinimization(symbol_manager, config)
nonlinear_minimizer.future_spread_mult = config.future_spread_mult
nonlinear_minimizer.lambda_reg = config.lambda_reg
plotly_manager = PlotlyManager(date_str, symbol_manager.fut_expiries)

print(f"✅ Pipeline ready | Options: {len(symbol_manager.opt_expiries)} expiries | Futures: {len(symbol_manager.fut_expiries)} expiries")

## 2. Data Processing & Analysis 📊

In [ ]:
# Data Conflation & Symbol Analysis 🔄
print(f"🔄 Conflating data (every {conflation_every}, period {conflation_period})...")
df_conflated_md = symbol_manager.get_conflated_md(freq=conflation_every, period=conflation_period).sort(['expiry', 'timestamp'])

options_breakdown = (symbol_manager.df_symbol.filter(pl.col('is_option'))
    .group_by(['expiry']).agg([pl.len().alias('total'), pl.col('strike').n_unique().alias('strikes')]))

print(f"✅ Conflated: {len(df_conflated_md):,} records")
print(f"📊 Options: {options_breakdown['total'].sum()} total across {len(symbol_manager.opt_expiries)} expiries")
print(f"🔮 Futures: {len(symbol_manager.fut_expiries)} expiries | Available: {symbol_manager.opt_expiries[:5]}...")

time_range = df_conflated_md.select([pl.col('timestamp').min().alias('start'), pl.col('timestamp').max().alias('end')]).to_dicts()[0]
print(f"⏰ Time range: {time_range['start']} → {time_range['end']}")

In [ ]:
# Single Expiry Option Chain Analysis 🎯
expiry = config.get('analysis.target_expiry', symbol_manager.opt_expiries[0])
year, month, day = int(date_str[:4]), int(date_str[4:6]), int(date_str[6:8])
timestamp = datetime(year, month, day, 0, 46, 30)

print(f"🎯 Analyzing: {expiry} @ {timestamp}")

try:
    df_option_chain, df_option_synthetic = symbol_manager.create_option_synthetic(df_conflated_md, expiry=expiry, timestamp=timestamp)
    print(f"✅ Created: {len(df_option_chain)} chain, {len(df_option_synthetic)} synthetic")
    
    if any('original_' in col for col in df_option_chain.columns):
        display(HTML(generate_price_comparison_table(df_option_chain, table_width="70%", font_size="10px")))
        print_tightening_effectiveness(calculate_tightening_stats(df_option_chain))
    
    if not df_option_synthetic.is_empty():
        display(df_option_synthetic.head())
    else:
        print("⚠️ No synthetic data")
        
except Exception as e:
    print(f"❌ Error: {e}")

## 3. Regression Analysis 📈

In [ ]:
# Regression Analysis (WLS + Constrained Optimization) 🧮
if 'df_option_synthetic' in locals() and not df_option_synthetic.is_empty():
    S, tau = df_option_synthetic['S'][0], df_option_synthetic['tau'][0]
    
    # Configure fitters ⚙️
    wls_regressor.set_printable(False)
    nonlinear_minimizer.set_printable(True)
    nonlinear_minimizer.reset_parameters(), nonlinear_minimizer.clear_results()
    
    print(f"�� Running regression on {len(df_option_synthetic)} observations...")
    
    # WLS regression 📏
    wls_result = wls_regressor.fit(df_option_synthetic, expiry=expiry, timestamp=timestamp)
    
    # Constrained optimization (if enabled) 🎯
    if use_constrained_optimization:
        try:
            initial_guess_const, initial_guess_coef = convert_rate_into_parameter((wls_result['r'], wls_result['q']), S, tau)
            final_result = nonlinear_minimizer.fit(df_option_synthetic, initial_guess_const, initial_guess_coef, expiry=expiry, timestamp=timestamp)
            method = "Constrained Optimization"
        except Exception as e:
            print(f"⚠️ Constrained optimization failed: {e}")
            final_result, method = wls_result, "WLS (fallback)"
    else:
        final_result, method = wls_result, "WLS Only"
    
    # Results 📋
    final_result['F'] = (nonlinear_minimizer if use_constrained_optimization else wls_regressor).get_implied_forward_price(final_result)
    basis_rate = (final_result['F'] / S - 1) * 100
    
    print(f"\n📈 RESULTS ({method}):")
    print(f"USD Rate (r): {final_result['r']:.6f} ({final_result['r']*100:.4f}%) | BTC Rate (q): {final_result['q']:.6f} ({final_result['q']*100:.4f}%)")
    print(f"Spread (r-q): {(final_result['r']-final_result['q'])*100:.4f}% | Forward: ${final_result['F']:.2f} | Basis: {basis_rate:.4f}%")
    print(f"R²: {final_result['r2']:.6f} | SSE: {final_result['sse']:.4f}")
else:
    print("❌ No synthetic data available")

In [ ]:
# Time Series Analysis - Condensed Demo 🚀
print(f"🚀 Time series | Interval: {time_interval_seconds}s | Smoothing λ: {old_weight}")

successful_fits = {}
fitter = nonlinear_minimizer if use_constrained_optimization else wls_regressor
wls_regressor.set_printable(False), nonlinear_minimizer.set_printable(False)

# Get time ranges for first 3 expiries (demo) 🎬
start_time_map = {each['expiry']: each for each in df_conflated_md.group_by('expiry').agg(
    pl.col('timestamp').first().alias('start_time'), pl.col('timestamp').last().alias('end_time')).to_dicts()}

for expiry in symbol_manager.opt_expiries[:3]:  # Limited for demo speed ⚡
    successful_fits[expiry] = {'total': 0, 'successful': 0}
    start_time = start_time_map[expiry]['start_time'] + timedelta(seconds=time_interval_seconds)
    end_time = start_time.replace(hour=23 if not symbol_manager.is_expiry_today(expiry) else 7, minute=59, second=0)
    print(f"start_time: {start_time}, end_time: {end_time}")
    timestamps = pl.datetime_range(start=start_time, end=end_time, interval=f"{time_interval_seconds}s", eager=True).to_list()
    
    initial_guess = None
    for ts in timestamps[:50]:  # First 50 timestamps for demo 📊
        try:
            df_chain, df_synthetic = symbol_manager.create_option_synthetic(df_conflated_md, expiry=expiry, timestamp=ts)
            if not df_synthetic.is_empty():
                tau, s0 = df_synthetic['tau'][0], df_synthetic['S'][0]
                is_cutoff, _ = fitter.check_if_cutoff_for_0DTE(expiry, ts, symbol_manager.is_expiry_today(expiry), s0, tau)
                
                if not is_cutoff:
                    successful_fits[expiry]['total'] += 1
                    if use_constrained_optimization:
                        if initial_guess is None or (np.isnan(initial_guess[0]) and np.isnan(initial_guess[1])):
                            wls_temp = wls_regressor.fit(df_synthetic, expiry=expiry, timestamp=ts)
                            initial_guess = (wls_temp['r'], wls_temp['q'])
                        initial_guess_const, initial_guess_coef = convert_rate_into_parameter(initial_guess, s0, tau)
                        result = fitter.fit(df_synthetic, initial_guess_const, initial_guess_coef, expiry=expiry, timestamp=ts)
                        initial_guess = (result['r'], result['q'])
                    else:
                        result = fitter.fit(df_synthetic, expiry=expiry, timestamp=ts)
                    
                    if result['success_fitting']: successful_fits[expiry]['successful'] += 1
        except Exception: continue
    
    success_rate = (successful_fits[expiry]['successful'] / successful_fits[expiry]['total'] * 100) if successful_fits[expiry]['total'] > 0 else 0
    print(f"✅ {expiry}: Successful Fit: {successful_fits[expiry]['successful']}/{successful_fits[expiry]['total']} ({success_rate:.1f}%)")

print("✅ Time series analysis complete (concise demo)")

In [ ]:
# Results DataFrame & Exponential Smoothing 📈
fit_result_manager = FitterResultManager(symbol_manager.opt_expiries, symbol_manager.fut_expiries, 
                                        symbol_manager.df_symbol, fitter.fit_results, successful_fits, old_weight=old_weight)

df_results = fit_result_manager.create_results_df(fit_result_manager.fit_results).sort('timestamp')

if not df_results.is_empty():
    print(f"✅ Generated {len(df_results):,} results with λ={old_weight} smoothing")
    
    # Summary stats 📊
    display(df_results.head(5))
    print(f"\n📊 Summary by expiry:")
    display(fit_result_manager.get_expiry_summary(df_results))
    
    # Smoothing info 🎚️
    print(f"\n🎚️ Smoothing: λ={old_weight} (old weight), 1-λ={1-old_weight} (new weight)")
else:
    print("❌ No results generated")

## 4. Visualization & Export 📊

In [ ]:
# Multi-Panel Visualization 📊
if not df_results.is_empty():
    # Create 4-panel plot 📈
    fig = make_subplots(rows=4, cols=1, 
                       subplot_titles=['USD Rate (r) %', 'BTC Rate (q) %', 'Rate Spread (r-q) %', 'Forward/Spot Ratio'],
                       vertical_spacing=0.08, shared_xaxes=True)
    
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']
    
    # Plot each expiry 🎨
    for i, exp in enumerate(symbol_manager.get_sorted_expiries()):
        exp_data = df_results.filter(pl.col('expiry') == exp).sort('timestamp')
        if exp_data.is_empty(): continue
        
        color, times = colors[i % len(colors)], exp_data['timestamp'].to_list()
        panels_data = [(exp_data['r'] * 100).to_list(), (exp_data['q'] * 100).to_list(),
                      (exp_data['smoothened_r-q'] * 100).to_list(), (exp_data['F'] / exp_data['S']).to_list()]
        
        for panel, data in enumerate(panels_data, 1):
            fig.add_trace(go.Scatter(x=times, y=data, mode='lines+markers', name=exp,
                line=dict(color=color, width=2), marker=dict(size=4),
                showlegend=(panel == 1), legendgroup=exp), row=panel, col=1)
    
    # Layout and show 🎪
    title_method = "Constrained Optimization" if use_constrained_optimization else "WLS Regression"
    fig.update_layout(title=f"{date_str} Bitcoin Options - {title_method} (λ={old_weight})",
        height=800, template='plotly_white')
    
    for i, label in enumerate(['USD Rate %', 'BTC Rate %', 'Rate Spread %', 'F/S Ratio'], 1):
        fig.update_yaxes(title_text=label, row=i, col=1)
    fig.update_xaxes(title_text="Time", row=4, col=1)
    
    fig.show()
    print("✅ Visualization complete!")
else:
    print("❌ No results to visualize")

In [ ]:
# Export Results 💾
if not df_results.is_empty():
    results_dir = os.path.join(project_root, "results", date_str)
    os.makedirs(results_dir, exist_ok=True)
    
    df_results.write_csv(os.path.join(results_dir, "baseoffset_results.csv"))
    df_conflated_md.write_csv(os.path.join(results_dir, "conflated_md.csv"))
    
    total_fits = len(df_results)
    successful_count = len(df_results.filter(pl.col('success_fitting') == True))
    success_rate = successful_count / total_fits if total_fits > 0 else 0
    
    print(f"📁 Exported: {total_fits:,} results ({success_rate:.1%} success) to {results_dir}")
    print(f"🎉 Analysis complete! Config: λ_reg={config.lambda_reg}, smoothing={old_weight}")
else:
    print("❌ No results to export")

## Summary 📋

**Bitcoin Options Analysis - Concise Version** 🚀

### ✅ **Completed Analysis**
- **Configuration**: YAML-driven parameters from `config/base_offset_config.yaml`
- **Data Processing**: Conflated market data with configurable intervals
- **Rate Extraction**: USD rate (r) and BTC rate (q) from put-call parity
- **Optimization**: WLS regression + constrained optimization with rate bounds
- **Smoothing**: Exponential smoothing with configurable λ parameter
- **Visualization**: Multi-panel time series with comprehensive analysis
- **Export**: Quality-controlled CSV export with success rate monitoring

### 🎯 **Key Benefits**
- **Streamlined Workflow**: All major analysis steps in under 40 lines per cell
- **Configuration-Driven**: All parameters externalized to YAML
- **Production Ready**: Same logic as main.py pipeline with comprehensive testing

Rate extraction, forward pricing, and arbitrage-free optimization with exponential smoothing for stable time series analysis.